**LOAD AND MERGE DATASETS**

In [ ]:
import pandas as pd

In [ ]:
sp=pd.read_csv('Spanish_merged_dataa.csv') # SPANISH DATASET

In [ ]:
sp.drop(['Unnamed: 0',	'Text',	'clean_with_stopwords'], axis=1, inplace=True)  # Drop unnecessary columns

In [ ]:
sp.rename(columns = {'Category':'Fake'}, inplace = True) #change column names

In [ ]:
fake=pd.read_csv('sp_true.csv') # TRANSLATED DATASET (TRUE)
real=pd.read_csv('sp_fake.csv') # TRANSLATED DATASET (FAKE)

In [ ]:
real.drop(['Unnamed: 0'], axis=1, inplace=True)  #Drop unnecessary columns

In [ ]:
real.rename(columns = {'final_clean_sp':'final_clean'}, inplace = True) #change column name

In [ ]:
fake.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
fake.rename(columns = {'text':'final_clean'}, inplace = True) #change column name

In [ ]:
data=pd.concat((fake,real,sp),axis=0) #CONCATENATE SPANISHFAKENEWS CORPUS

In [ ]:
data

,Fake,final_clean
0,0,Intenta prender fuego a su esposa: ella está i...
1,0,"¡Eso es, Renaud está de vuelta en el escenario..."
2,0,"Con su traje rojo, ¿qué mensaje quería enviar ..."
3,0,Foot-Ligue 1: todos los resultados de la octav...
4,0,× Suscríbete al podcast\nPuede suscribirse a l...
...,...,...
2966,0,consejo gobierno haber dar visto bueno convoca...
2967,0,investigador valenciano haber desarrollar inno...
2968,0,arrestado actuar coche establecimiento policia...
2969,0,rey haber encargar miercoles pedro sanchez pre...


**COUNT NUMBER OF FAKE AND REAL NEWS ARTICLES**

In [ ]:
data['Fake'].value_counts()

1    3000
0    3000
Name: Fake, dtype: int64

**CONVERT TO LOWERCASE**

In [ ]:
data['final_clean'] = data['final_clean'].str.lower()  # CONVERT ARTICLES TO LOWERCASE

**REMOVE STOPWORDS**

In [ ]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=b49c6bc481343512d4646d2a920dfad7d84885cfdef7b9a1abb4b6ef32dc4b1e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))   #IMPORT SPANISH STOPWORDS

In [ ]:
def remove_stop_words(sentence):   #REMOVE SPANISH STOPWORDS
    stop_words  = set(stopwords.words('spanish'))
    word_list=sentence.split()
    clean_sentence=' '.join([w for w in word_list if w not in stop_words])
    return(clean_sentence) 

In [ ]:
data['final_clean'] = data['final_clean'].apply(remove_stop_words)

**LEMMATIZE WORDS**

In [ ]:
!pip install -U pip setuptools wheel

     |████████████████████████████████| 1.7 MB 24.8 MB/s 
     |████████████████████████████████| 952 kB 42.2 MB/s 
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.0
    Uninstalling wheel-0.37.0:
      Successfully uninstalled wheel-0.37.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install -U spacy

     |████████████████████████████████| 6.0 MB 31.0 MB/s            
     |████████████████████████████████| 181 kB 20.4 MB/s            
     |████████████████████████████████| 10.1 MB 37.7 MB/s            
     |████████████████████████████████| 451 kB 44.5 MB/s            
     |████████████████████████████████| 628 kB 34.2 MB/s            
     |████████████████████████████████| 42 kB 1.1 MB/s             
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully un

In [ ]:
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 14.0 MB 10.2 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
import spacy
from spacy import displacy

In [ ]:
nlp = spacy.load('es_core_news_sm')  

In [ ]:
data['final_clean'] = data['final_clean'].apply(lambda x: " ".join([y.lemma_ for y in nlp(x)])) #LEMMATIZE TEXT

**REMOVE PUNCTUATION AND NON ALPHANUMERIC CHARACTERS**

In [ ]:
import re 

def clean_data(text):
    
    text = re.sub(r'<br />', ' ', text) #Removes Html tag
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ',text)
    text = re.sub(r'[^\ a-zá-ú]+', '', text)  #Removes non-alphabetic characters
    text = re.sub(r'^\s*|\s\s*', ' ', text).strip() #Removes extra whitespace, tabs
    return text

In [ ]:
data['final_clean'] = data['final_clean'].apply(lambda x: clean_data(x))

**SAVE TO .CSV FORMAT**

In [ ]:
data.to_csv('sp_translated.csv')

In [ ]:
data.head()

,Fake,final_clean
0,0,intentar prender fuego espós ilesa gravemente ...
1,0,ese ser renaud vuelta escenario octubre acción...
2,0,traje rojo qué mensaje querer enviar hillary c...
3,0,footligar resultado octava jornada h octubre m...
4,0,suscríbete podcast poder suscribir él descarga...
